In [10]:
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.svm import SVC

In [11]:
models = []
models.append(('LinearDiscriminantAnalysis', LinearDiscriminantAnalysis()))
models.append(('SVC', SVC(random_state=1)))

In [12]:
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.preprocessing import MinMaxScaler, Normalizer,\
StandardScaler, RobustScaler, LabelEncoder, FunctionTransformer

transformers = []
scaler = StandardScaler()
transformers.append(('scaler', scaler))
t1 = FeatureUnion(transformers)
t1

FeatureUnion(n_jobs=1,
       transformer_list=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))],
       transformer_weights=None)

In [13]:
from sklearn.decomposition import PCA
transformers = []
pca = PCA(n_components=2, svd_solver='randomized', whiten=True)
transformers.append(('pca', pca))
t2 = FeatureUnion(transformers)
t2

FeatureUnion(n_jobs=1,
       transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=2, random_state=None,
  svd_solver='randomized', tol=0.0, whiten=True))],
       transformer_weights=None)

In [14]:
pipelines = []
for m in models:
    pipelines.append((m[0],
        Pipeline([
            ('preparer', t1),
            ('selector', t2),
            m,
        ])
    ))

In [30]:
pipelines

[('LinearDiscriminantAnalysis', Pipeline(memory=None,
       steps=[('preparer', FeatureUnion(n_jobs=1,
         transformer_list=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))],
         transformer_weights=None)), ('selector', FeatureUnion(n_jobs=1,
         transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=2, ran...None, priors=None, shrinkage=None,
                solver='svd', store_covariance=False, tol=0.0001))])),
 ('SVC', Pipeline(memory=None,
       steps=[('preparer', FeatureUnion(n_jobs=1,
         transformer_list=[('scaler', StandardScaler(copy=True, with_mean=True, with_std=True))],
         transformer_weights=None)), ('selector', FeatureUnion(n_jobs=1,
         transformer_list=[('pca', PCA(copy=True, iterated_power='auto', n_components=2, ran...bf',
    max_iter=-1, probability=False, random_state=1, shrinking=True,
    tol=0.001, verbose=False))]))]

In [31]:
from sklearn.model_selection import KFold
kfold = KFold(n_splits=10, random_state=7)
kfold

KFold(n_splits=10, random_state=7, shuffle=False)

In [32]:
from sklearn.model_selection import train_test_split
from sklearn import datasets, linear_model
x = datasets.load_iris().data
y = datasets.load_iris()
y = y.target

In [33]:
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2)

In [34]:
from sklearn.model_selection import cross_val_score
for name, model in pipelines:
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring='accuracy')
cv_results

array([ 1.        ,  1.        ,  0.66666667,  0.91666667,  1.        ,
        1.        ,  0.91666667,  0.91666667,  1.        ,  0.83333333])

In [35]:
print(cv_results.mean())
print(cv_results.std())

0.925
0.101721296798
